In [ ]:
#安装OpenAI
!pip install openai


In [ ]:
#设置你的apiKey和baseURL
apikey="Ur KEY"
base_url="https://api.deepseek.com"#这里用的是DeepSeek API，需要换成其他源可以自行更改
model = "deepseek-chat"

In [ ]:
from openai import OpenAI
import time


def entityExtract(sentence: str):
    client = OpenAI(api_key=apikey, base_url=base_url)
    model = model
    prompt = """
    对###之后的这段话进行实体提取和关系提取，并使用Json格式进行输出，每个实体都是单独的条目，例如{"name":“赤眼蜂”,"tag"：“益虫”}，关系用三元组表示一组关系，例如{
          "主体": "麦蚜",
          "谓词": "属于",
          "客体": "虫害"
        }{
          "主体": "稻瘟病",
          "谓词": "属于",
          "客体": "病害"
        }实体和关系要在json中分开，例如{
        "实体":[],
        "关系":[]
        }：###\n
    """
    sysPrompt = "你是一个乐于助人的智能助手。"
    start_time = time.time()
    Messages = [
        {"role": "system", "content": sysPrompt},
        {"role": "user", "content": prompt + sentence},
    ]
    response = client.chat.completions.create(model=model, messages=Messages, stream=False)

    end_time = time.time()
    executionTime = end_time - start_time
    # print(f"代码块执行时间: {executionTime} 秒")
    # print("Token 消耗数量:")
    # print(response.usage)
    return response.choices[0].message.content


def makeSentence(sentence: str):
    client = OpenAI(api_key=apikey, base_url=base_url)
    model = model
    # prompt= """
    # 在###后是用json格式表示的两个实体以及一个关系描述，其中"实体1"代表主体，"实体2"代表客体，"name"代表这个实体的名称，"tag"用于辅助描述这个实体，有时候在"关系"中的“谓语”为英文，请你把它翻译成中文，我需要你利用这些信息来造五个句子，不必每一个句子都包含两个实体或者包含完整的关系，但是造的句子一定要贴合我给出的信息，不要对实体和三元组评价，只需要利用这些信息来造句子，句子长短不限，为了方便我解析你的回复，请用list来表示这些句子###
    # """+str(sentence)
    # prompt= """
    # 在###后是用json格式表示的两个实体以及一个关系描述，其中"实体1"代表主体，"实体2"代表客体，"name"代表这个实体的名称，"tag"用于辅助描述这个实体，有时候在"关系"中的“谓语”为英文，请你将其视为中文使用，我需要你利用这些信息来造五个句子，为了方便我解析你的回复，请用list来表示这些句子###
    # """+str(sentence)
    prompt = """
    在###后是用json格式表示的两个实体以及一个关系描述，其中"实体1"代表主体，"实体2"代表客体，"name"代表这个实体的名称，"tag"用于辅助描述这个实体，有时候在"关系"中的“谓语”为英文，请你将其视为中文使用，我需要你利用这些信息来写一段话。这段话在语言上要体现你就好像本来就知道###后的信息一样，不要在对话中强调json的key，比如xxx是xxx的客体，xxx与xxx具有xxx关系，这里的xxx指的是json中的value，这样会显得你是被要求这么说的，这是一个举例：给定的json：{
          "主体": "麦蚜",
          "谓词": "属于",
          "客体": "虫害"
        }，造句:"麦蚜是病虫害的一种"，而不是："麦蚜属于虫害";"麦蚜是主体";"虫害是客体"。
        这段话要能完整体现json中的信息###
    """ + str(sentence)
    sysPrompt = "你是一个乐于助人的智能助手。"

    Messages = [
        {"role": "system", "content": sysPrompt},
        {"role": "user", "content": prompt},
    ]
    start_time = time.time()
    response = client.chat.completions.create(model=model, messages=Messages, stream=False)

    end_time = time.time()
    executionTime = end_time - start_time
    # print(f"代码块执行时间: {executionTime} 秒")
    # print("Token 消耗数量:")
    # print(response.usage)
    return response.choices[0].message.content


def parser(string: str):
    string = string.replace("\n", '').replace(" ", "")
    string = string[1:]
    string = string[:-1].replace('\"', '')
    return string.split(",")


def generate(source):
    content = makeSentence(source).replace('\n', '')
    res = entityExtract(content).replace('\n', '').replace('```json', '').replace("```", '').replace(" ", '')

    # print(res)
    datasetTemp = {
        "source": source,
        "sentence": content,
        "result": res
    }
    return datasetTemp

In [ ]:
import json
with open('./output3.txt','r',encoding='utf8') as f:
    data=json.load(f)
print(len(data))
with open('./datasetTemp.txt','a',encoding='utf8') as f:
    f.write('[')
    jumpto=0#用于恢复进度
    counter=0
    for i in data[5]:
        if counter<jumpto:

            counter+=1
            continue
        while True:
            try:
                datatemp = generate(i)
                json.dump(datatemp, f, ensure_ascii=False)
                f.write(',')
                counter += 1
                print("计数器:" + str(counter))
                break  # 如果没有异常，跳出while循环
            except Exception as e:
                print(f"生成数据时发生错误: {e}")
                if 'code: 400' in str(e):
                    break
                print("正在重新尝试...")

    f.write(']')